In [1]:
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 21.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 51.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
import re
import copy
import pandas as pd
import pyarrow
from tqdm import tqdm
import os
from collections import Counter
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import pyarrow.parquet as pq

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/datalore/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
stop_words = set(stopwords.words('english'))

In [5]:
# get file path as dict[int, list[str]] where 
# key is the year and value is the list of file paths

# break the text into itemized sections
def get_itemized_10k(fname, sections=['business', 'risk', 'mda', 'qmr']):
    '''Extract ITEM from 10k filing text.

    Args:
        fname: str, the file name (ends with .txt)
        sections: list of sections to extract

    Returns:
        itemized_text: dict[str, str], where key is the section name and value is the text
    '''
    
    with open(fname, encoding='utf-8') as f:
        text = f.read()

    def extract_text(text, item_start, item_end):
        '''
        Args:
            text: 10K filing text
            item_start: compiled regex pattern
            item_end: compiled regex pattern
        '''
        item_start = item_start
        item_end = item_end
        starts = [i.start() for i in item_start.finditer(text)]
        ends = [i.start() for i in item_end.finditer(text)]

        # if no matches, return empty string
        if len(starts) == 0 or len(ends) == 0:
            return None

        # get possible start/end positions
        # we may end up with multiple start/end positions, and we'll choose the longest
        # item text.
        positions = list()
        for s in starts:
            control = 0
            for e in ends:
                if control == 0:
                    if s < e:
                        control = 1
                        positions.append([s,e])

        # get the longest item text
        item_length = 0
        item_position = list()
        for p in positions:
            if (p[1]-p[0]) > item_length:
                item_length = p[1]-p[0]
                item_position = p

        item_text = text[item_position[0]:item_position[1]]

        return item_text

    # extract text for each section
    results = {}

    for section in sections:

        # ITEM 1: Business
        # if there's no ITEM 1A then it ends at ITEM 2
        if section == 'business':
            try:
                item1_start = re.compile("i\s?tem[s]?\s*[1I]\s*[\.\;\:\-\_]*\s*\\b|PART\s*[1I]\s\s*[\.\;\:\-\_]*\s*", re.IGNORECASE)
                item1_end = re.compile("item\s*1a\s*[\.\;\:\-\_]*\s*Risk|item\s*2\s*[\.\,\;\:\-\_]*\s*(Desc|Prop)", re.IGNORECASE)
                business_text = extract_text(text, item1_start, item1_end)
                results['business'] = business_text
            except Exception as e:
                results['business'] = ''
                print(f'Error extracting ITEM 1: Business for {fname}')
            
        # ITEM 1A: Risk Factors
        # it ends at ITEM 2
        if section == 'risk':
            try:
                item1a_start = re.compile("(?<!,\s)item\s*1a[\.\;\:\-\_]*\s*Risk", re.IGNORECASE)
                item1a_end = re.compile("item\s*2\s*[\.\;\:\-\_]*\s*(Desc|Prop)|item\s*[1I]\s*[\.\;\:\-\_]*\s*\\b", re.IGNORECASE)
                risk_text = extract_text(text, item1a_start, item1a_end)
                results['risk'] = risk_text
            except Exception as e:
                results['risk'] = ''
                print(f'Error extracting ITEM 1A: Risk Factors for {fname}')
                
        # ITEM 7: Management's Discussion and Analysis of Financial Condition and Results of Operations
        # it ends at ITEM 7A (if it exists) or ITEM 8
        if section == 'mda':
            try:
                item7_start = re.compile("item\s*7\s*[\.\;\:\-\_]*\s*\\bM", re.IGNORECASE)
                item7_end = re.compile("item\s*7a\s*[\.\;\:\-\_]*[\s\n]*Quanti|item\s*8\s*[\.\,\;\:\-\_]*\s*Finan", re.IGNORECASE)
                item7_text = extract_text(text, item7_start, item7_end)
                results['mda'] = item7_text
            except Exception as e:
                results['mda'] = ''
                print(f'Error extracting ITEM 7: MD&A for {fname}')

        # ITEM 7A: Quantitative and Qualitative Disclosures About Market Risk
        # 
        if section == 'qmr':
            try:
                item7a_start = re.compile("item\s*7a\s*[\.\;\:\-\_]*[\s\n]*Quanti", re.IGNORECASE)
                item7a_end = re.compile("item\s*8\s*[\.\,\;\:\-\_]*\s*Finan", re.IGNORECASE)
                item7a_text = extract_text(text, item7a_start, item7a_end)
                results['qmr'] = item7a_text
            except Exception as e:
                results['qmr'] = ''
                print(f'Error extracting ITEM 7A: for {fname}')
    
    return results


def get_itemized_10q(fname, sections=['mda', 'qmr']):
    '''Extract ITEM from 10q filing text.
    Args:
        fname: str, the file name (ends with .txt)
        sections: list of sections to extract
    Returns:
        itemized_text: dict[str, str], where key is the section name and value is the text
    '''
    
    with open(fname, 'r') as f:
        text = f.read()

    def extract_text(text, item_start, item_end):
        '''
        Args:
            text: 10q filing text
            item_start: compiled regex pattern
            item_end: compiled regex pattern
        '''
        item_start = item_start
        item_end = item_end
        starts = [i.start() for i in item_start.finditer(text)]
        ends = [i.start() for i in item_end.finditer(text)]

        # if no matches, return empty string
        if len(starts) == 0 or len(ends) == 0:
            return None

        # get possible start/end positions
        # we may end up with multiple start/end positions, and we'll choose the longest
        # item text.
        positions = list()
        for s in starts:
            control = 0
            for e in ends:
                if control == 0:
                    if s < e:
                        control = 1
                        positions.append([s,e])

        # get the longest item text
        item_length = 0
        item_position = list()
        for p in positions:
            if (p[1]-p[0]) > item_length:
                item_length = p[1]-p[0]
                item_position = p

        item_text = text[item_position[0]:item_position[1]]

        return item_text

    # extract text for each section
    results = {}

    for section in sections:

        # ITEM 2: Management's Discussion and Analysis of Financial Condition and Results of Operations
        # it ends at ITEM 3
        if section == 'mda':
            try:
                item2_start = re.compile("item\s*2\s*[\.\;\:\-\_]*[\s\n]*Man", re.IGNORECASE)
                item2_end = re.compile("item\s*3\s*[\.\;\:\-\_]*[\s\n]*Quanti|item\s*4\s*", re.IGNORECASE)
                item2_text = extract_text(text, item2_start, item2_end)
                results['mda'] = item2_text
            except Exception as e:
                results['mda'] = ''
                print(f'Error extracting ITEM 2: MD&A for {fname}')
        
        if section == 'qmr':
            try:
                item7a_start = re.compile("item\s*3\s*[\.\;\:\-\_]*[\s\n]*Quanti", re.IGNORECASE)
                item7a_end = re.compile("item\s*4", re.IGNORECASE)
                item7a_text = extract_text(text, item7a_start, item7a_end)
                results['qmr'] = item7a_text
            except Exception as e:
                results['qmr'] = ''
                print(f'Error extracting ITEM 7A: for {fname}')


    return results

In [6]:
def get_info_from_file_name(file_path):
    info = file_path.replace('_edgar', '').replace('_data', '').replace('.txt', '').split('_')
    #['20110328', '10-K', '1403720', '0001469299-11-000174'] 
    date = info[0]
    form_type = info[1]
    cik = info[2]
    acc_num = info[3]

    return date, form_type, cik, acc_num

In [7]:
def word_freq(text):
    if text is None:
        return {}
    text = text.replace('\n', ' ').replace('\t', ' ')
    words = text.split(' ')
    count = {}
    for word in words:
        word = word.replace("'", "").replace('"', '').replace('(', '').replace(')', '').replace('#', '').replace(',','')
        if word.isalpha():
            if word.lower() not in stop_words:
                count[word] = count.get(word, 0) + 1
    return count

In [8]:
word_freq('This is an apple apple')

{'apple': 2}

In [9]:
sections_10k = ['business', 'risk', 'mda', 'qmr']
sections_10q = ['mda', 'qmr']
parquet_path = '/data/workspace_files/section_dict/'
for year in range(2011, 2021):
    #print(yr)
    yr = str(year)
    for qtr in ['QTR1', 'QTR2', 'QTR3', 'QTR4']:
        print(f"Running Year {yr}, QTR : {qtr}")
        #print(qtr)
        clean_text_folder = '/data/workspace_files/10X_folders/' + yr + '/' + qtr
        files = [f for f in os.listdir(clean_text_folder)]
        for idx in range(0, len(files), 500):
            save_name = parquet_path + yr + '_' + qtr + '_' + str(idx) + '.parquet'
            if os.path.exists(save_name):
                print(f"Exists : {save_name}")
                continue
            df_list = []
            pbar = tqdm(files[idx:idx + 500])
            for file_path in pbar:
                # print(file_path)
                date, form_type, cik, acc_num = get_info_from_file_name(file_path)
                if form_type != '10-K' and form_type != '10-Q':
                    continue
                file_path_req = clean_text_folder + '/' + file_path
                if form_type == '10-K':
                    doc_dict = get_itemized_10k(file_path_req)
                    keys = copy.deepcopy(sections_10k)
                else:
                    doc_dict = get_itemized_10q(file_path_req)
                    keys = copy.deepcopy(sections_10q)
                row = {'Date': date, "Form_Type": form_type, 'CIK': cik, "Accession_Number": acc_num}
                for key in keys:
                    row[key] = str(word_freq(doc_dict[key]))
                df_list.append(row)
            df = pd.DataFrame(df_list)
            df.to_parquet(save_name, index=False)
            print("Done for", save_name)
        print("Done for qtr", qtr)
        #break
    #break
    print("Done for year", yr)

Running Year 2011, QTR : QTR1
Exists : /data/workspace_files/section_dict/2011_QTR1_0.parquet
Exists : /data/workspace_files/section_dict/2011_QTR1_500.parquet
Exists : /data/workspace_files/section_dict/2011_QTR1_1000.parquet
Exists : /data/workspace_files/section_dict/2011_QTR1_1500.parquet
Exists : /data/workspace_files/section_dict/2011_QTR1_2000.parquet
Exists : /data/workspace_files/section_dict/2011_QTR1_2500.parquet
Exists : /data/workspace_files/section_dict/2011_QTR1_3000.parquet
Exists : /data/workspace_files/section_dict/2011_QTR1_3500.parquet
Exists : /data/workspace_files/section_dict/2011_QTR1_4000.parquet
Exists : /data/workspace_files/section_dict/2011_QTR1_4500.parquet
Exists : /data/workspace_files/section_dict/2011_QTR1_5000.parquet
Exists : /data/workspace_files/section_dict/2011_QTR1_5500.parquet
Exists : /data/workspace_files/section_dict/2011_QTR1_6000.parquet
Exists : /data/workspace_files/section_dict/2011_QTR1_6500.parquet
Exists : /data/workspace_files/secti

100%|██████████| 309/309 [00:13<00:00, 22.37it/s]


In [15]:
parquet_path = '/data/workspace_files/section_dict/'
file_name = f'{2011}_QTR{1}_{0}.parquet'
file_path_read_req = parquet_path + file_name
test = pq.read_table(file_path_read_req).to_pandas()

In [14]:
test.iloc[-1]['business']

"{'PART': 1, 'ITEM': 1, 'General': 1, 'Worlds': 40, 'leading': 5, 'entertainment': 6, 'portal': 2, 'leverages': 1, 'proprietary': 3, 'technology': 24, 'offer': 4, 'visitors': 2, 'network': 1, 'virtual': 20, 'environments': 3, 'call': 1, 'worlds': 13, 'visually': 1, 'engaging': 1, 'online': 13, 'featuring': 3, 'animation': 2, 'motion': 4, 'content': 13, 'people': 2, 'come': 1, 'together': 1, 'navigating': 1, 'website': 2, 'shop': 2, 'interact': 3, 'others': 2, 'attend': 1, 'events': 1, 'support': 1, 'overall': 1, 'business': 4, 'strategy': 3, 'design': 2, 'develop': 8, 'software': 4, 'related': 7, 'creation': 3, 'interactive': 13, 'Internet': 13, 'web': 6, 'Using': 2, 'create': 4, 'sites': 13, 'well': 7, 'available': 3, 'service': 6, 'Sites': 1, 'using': 3, 'allow': 2, 'numerous': 4, 'simultaneous': 1, 'enter': 5, 'navigate': 1, 'share': 3, 'designed': 2, 'promote': 1, 'frequent': 2, 'repeat': 2, 'prolonged': 1, 'visitation': 1, 'users': 17, 'providing': 1, 'unique': 3, 'communities': 2

In [15]:
test.columns

Index(['Date', 'Form_Type', 'CIK', 'Accession_Number', 'business', 'risk',
       'mda', 'qmr'],
      dtype='object')

In [10]:
def create_section_dict_year(section, year, req_columns):
    parquet_path = '/data/workspace_files/section_dict/'
    save_name = f'{parquet_path}{section}/{year}.parquet'
    year_df = []
    for qtr in range(1, 5):
        print(f'Running for QTR {qtr}')
        for file_idx in range(0, 20000, 500):
            file_name = f'{year}_QTR{qtr}_{file_idx}.parquet'
            file_path_read_req = parquet_path + file_name
            try: 
                df = pq.read_table(file_path_read_req, columns = req_columns + [section]).to_pandas()
            except Exception as e:
                print(e)
                break
            year_df.append(df)
    
    year_df = pd.concat(year_df, axis=0)
    year_df.to_parquet(save_name, index=False)

    return 

In [11]:
req_columns = ['Date', 'Form_Type', 'CIK', 'Accession_Number']
for section in ['business', 'risk', 'mda', 'qmr']:
    for year in range(2011, 2021):
        create_section_dict_year(section, str(year), req_columns)

Running for QTR 1
/data/workspace_files/section_dict/2011_QTR1_9500.parquet
Running for QTR 2
/data/workspace_files/section_dict/2011_QTR2_11000.parquet
Running for QTR 3
/data/workspace_files/section_dict/2011_QTR3_11000.parquet
Running for QTR 4
/data/workspace_files/section_dict/2011_QTR4_10000.parquet
Running for QTR 1
/data/workspace_files/section_dict/2012_QTR1_9000.parquet
Running for QTR 2
/data/workspace_files/section_dict/2012_QTR2_10500.parquet
Running for QTR 3
/data/workspace_files/section_dict/2012_QTR3_10000.parquet
Running for QTR 4
/data/workspace_files/section_dict/2012_QTR4_9500.parquet
Running for QTR 1
/data/workspace_files/section_dict/2013_QTR1_8000.parquet
Running for QTR 2
/data/workspace_files/section_dict/2013_QTR2_10500.parquet
Running for QTR 3
/data/workspace_files/section_dict/2013_QTR3_9000.parquet
Running for QTR 4
/data/workspace_files/section_dict/2013_QTR4_8500.parquet
Running for QTR 1
/data/workspace_files/section_dict/2014_QTR1_8500.parquet
Runnin